In [10]:
def find_timer_settings(apb_clk, target_freq,
                        max_prescaler=65535,
                        max_period=65535,
                        rel_tol=1e-6):
    """
    找出所有滿足
        Freq = apb_clk / ((PSC+1)*(ARR+1))
    的 PSC, ARR 組合，使得計算頻率與目標頻率誤差在相對容差 rel_tol 以內。

    參數:
      apb_clk      -- APB 時鐘頻率 (Hz)
      target_freq  -- 期望的定時器輸出頻率 (Hz)
      max_prescaler-- PSC 最大值 (預設 65535)
      max_period   -- ARR 最大值 (預設 65535)
      rel_tol      -- 相對誤差容差 (預設 1e-6)

    回傳:
      一個列表，元素為 (PSC, ARR, achieved_freq, error)
    """
    results = []
    for psc in range(max_prescaler + 1):
        # 由公式反推 ARR + 1
        arr_f = apb_clk / ((psc + 1) * target_freq) - 1
        if not (0 <= arr_f <= max_period):
            continue

        # 四捨五入到最近整數
        arr = int(round(arr_f))
        if not (0 <= arr <= max_period):
            continue

        # 計算實際頻率及誤差
        freq = apb_clk / ((psc + 1) * (arr + 1))
        err = abs(freq - target_freq)
        if err / target_freq <= rel_tol:
            results.append((psc, arr, freq, err))

    return results

if __name__ == "__main__":
    APB_CLK    = 150_000_000
    TARGET_HZ  = 10_000_000
    solutions = find_timer_settings(APB_CLK, TARGET_HZ)

    print(f"找到 {len(solutions)} 種可行組合：")
    for psc, arr, freq, err in solutions:
        print(f"  PSC={psc:5d}, ARR={arr:5d} → Freq={freq:10.4f} Hz (誤差 {err:.4e} Hz)")


找到 4 種可行組合：
  PSC=    0, ARR=   14 → Freq=10000000.0000 Hz (誤差 0.0000e+00 Hz)
  PSC=    2, ARR=    4 → Freq=10000000.0000 Hz (誤差 0.0000e+00 Hz)
  PSC=    4, ARR=    2 → Freq=10000000.0000 Hz (誤差 0.0000e+00 Hz)
  PSC=   14, ARR=    0 → Freq=10000000.0000 Hz (誤差 0.0000e+00 Hz)
